In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

# --- 関数1：エリア内の全店舗URLを取得する ---
def get_all_shop_urls_from_area(area_top_url):
    """
    指定されたエリアの店舗一覧を全ページ巡回し，
    全店舗のURLと店名をリストで返す関数
    """
    shop_list = []
    page = 1
    
    while True:
        if page == 1:
            target_url = area_top_url
        else:
            target_url = f"{area_top_url.rstrip('/')}/rstLst/{page}/"
            
        print(f"店舗一覧の{page}ページ目を収集中...")
        
        try:
            response = requests.get(target_url)
            if response.status_code != 200:
                print("-> 店舗一覧の次のページが見つかりませんでした．")
                break
            
            soup = BeautifulSoup(response.content, 'html.parser')
            shop_tags = soup.select('a.list-rst__rst-name-target')
            
            if not shop_tags:
                print("-> このページに店舗情報がありませんでした．")
                break

            for tag in shop_tags:
                shop_list.append({
                    'name': tag.get_text(strip=True),
                    'url': tag['href']
                })

            page += 1
            time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"エラーが発生しました: {e}")
            break
            
    return shop_list

# --- 関数2：1店舗の全口コミを取得する ---
def get_all_reviews_from_shop(shop_url):
    """1つの店舗URLから，全ての口コミ本文を取得する"""
    
    shop_reviews = []
    page_num = 1
    review_base_url = f"{shop_url}dtlrvwlst/COND-0/smp1/?smp=1&lc=0&rvw_part=all"

    while True:
        if page_num == 1:
            target_url = review_base_url
        else:
            target_url = f"{review_base_url}&PG={page_num}"
        
        print(f"  -> 口コミの{page_num}ページ目を収集中...")
        
        try:
            response = requests.get(target_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            review_elements = soup.find_all('div', class_='rvw-item__rvw-comment')

            if not review_elements:
                break

            for review in review_elements:
                shop_reviews.append(review.get_text(strip=True))

            page_num += 1
            time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"    口コミ取得中にエラー: {e}")
            break
            
    return shop_reviews

# --- メインの実行処理 ---
if __name__ == '__main__':
    
    # 1. お台場エリアのトップページのURLを指定
    odaiba_area_url = 'https://tabelog.com/tokyo/A1313/A131306/'
    
    # 2. 関数1を呼び出し，お台場の全店舗リストを取得
    print("--- ステップ1：お台場の店舗リスト収集開始 ---")
    all_shops_in_odaiba = get_all_shop_urls_from_area(odaiba_area_url)
    print(f"\n収集完了．お台場エリアで {len(all_shops_in_odaiba)}軒の店舗が見つかりました．\n")
    
    # 3. CSVファイルを用意し，各店舗の口コミを収集・保存
    print("--- ステップ2：各店舗の口コミ収集開始 ---")
    output_filename = 'odaiba_reviews2.csv'
    
    with open(output_filename, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['shop_name', 'shop_url', 'review_text'])
        
        # 4. 取得した店舗リストでループを開始
        for shop in all_shops_in_odaiba:
            shop_name = shop['name']
            shop_url = shop['url']
            print(f"\n【{shop_name}】の口コミを取得します...")
            
            # 5. 関数2を呼び出し，その店の全口コミを取得
            reviews = get_all_reviews_from_shop(shop_url)
            
            print(f"  -> {len(reviews)}件の口コミを取得しました．CSVに書き込みます．")
            
            # 6. 取得した口コミを1行ずつCSVに書き込み
            for review_text in reviews:
                writer.writerow([shop_name, shop_url, review_text])
            
            # 7. 次の店舗に移る前に長めに待機（重要！）
            print("  -> 3秒待機します...")
            time.sleep(3)

    print(f"\n全ての処理が完了しました．「{output_filename}」を確認してください．")

--- ステップ1：お台場の店舗リスト収集開始 ---
店舗一覧の1ページ目を収集中...
店舗一覧の2ページ目を収集中...
店舗一覧の3ページ目を収集中...
店舗一覧の4ページ目を収集中...
店舗一覧の5ページ目を収集中...
店舗一覧の6ページ目を収集中...
店舗一覧の7ページ目を収集中...
店舗一覧の8ページ目を収集中...
店舗一覧の9ページ目を収集中...
店舗一覧の10ページ目を収集中...
店舗一覧の11ページ目を収集中...
店舗一覧の12ページ目を収集中...
店舗一覧の13ページ目を収集中...
店舗一覧の14ページ目を収集中...
店舗一覧の15ページ目を収集中...
店舗一覧の16ページ目を収集中...
店舗一覧の17ページ目を収集中...
店舗一覧の18ページ目を収集中...
店舗一覧の19ページ目を収集中...
店舗一覧の20ページ目を収集中...
店舗一覧の21ページ目を収集中...
店舗一覧の22ページ目を収集中...
店舗一覧の23ページ目を収集中...
店舗一覧の24ページ目を収集中...
店舗一覧の25ページ目を収集中...
店舗一覧の26ページ目を収集中...
店舗一覧の27ページ目を収集中...
店舗一覧の28ページ目を収集中...
店舗一覧の29ページ目を収集中...
店舗一覧の30ページ目を収集中...
店舗一覧の31ページ目を収集中...
店舗一覧の32ページ目を収集中...
店舗一覧の33ページ目を収集中...
店舗一覧の34ページ目を収集中...
店舗一覧の35ページ目を収集中...
店舗一覧の36ページ目を収集中...
店舗一覧の37ページ目を収集中...
店舗一覧の38ページ目を収集中...
店舗一覧の39ページ目を収集中...
店舗一覧の40ページ目を収集中...
店舗一覧の41ページ目を収集中...
店舗一覧の42ページ目を収集中...
店舗一覧の43ページ目を収集中...
店舗一覧の44ページ目を収集中...
店舗一覧の45ページ目を収集中...
店舗一覧の46ページ目を収集中...
店舗一覧の47ページ目を収集中...
店舗一覧の48ページ目を収集中...
店舗一覧の49ページ目を収集中...
店舗一覧の50ページ目を収集中...
店舗一覧の51ページ目を収集中...
店舗一覧の52ページ目を